# CDCS Data Management

This Notebook details the functions and interactions for managing data in a CDCS instance.  Primarily, this means having an account on the database allowing you to log in to create and modify records. 

Notes on CDCS design:

- Each record is assigned to a user and can be assigned to a workspace.
- Any records without a workspace are only accessible to the assigned user.
- A workspace serves as a group of records that can be made accessible to specific users.
- Each CDCS instance has a "Global Public Workspace" that is meant for all users to be able to see.

In [1]:
from pathlib import Path

import cdcs
from cdcs import CDCS

print('Notebook executed for cdcs version', cdcs.__version__)

Notebook executed for cdcs version 0.1.9


## 1. Class initialization

A CDCS client manager can be initialized by passing it the host url and authenication information.

Parameters

- __host__: (*str*) URL for the database's server.
- __username__: (*str, optional*) Username of desired account on the server. A prompt will ask for the username if not given.
- __password__: (*str, optional*) Password of desired account on the server.  A prompt will ask for the password if not given.
- __auth__: (*tuple, optional*) Auth tuple to enable Basic/Digest/Custom HTTP Auth.  Alternative to giving username and password seperately.
- __cert__: (*str, optional*) if String, path to ssl client cert file (.pem). If Tuple, (‘cert’, ‘key’) pair.
- __certification__: (*str, optional*) Alias for cert. Retained for compatibility.
- __verify__: (*bool or str, optional*) Either a boolean, in which case it controls whether we verify the server’s TLS certificate, or a string, in which case it must be a path to a CA bundle to use. Defaults to True.

In [2]:
curator = CDCS('https://potentials.nist.gov/', username='lmh1')

Enter password for lmh1 @ https://potentials.nist.gov:········


## 2. Query data

The query() method will return *all* matching records that you have access to.

Parameters

- __template__: (*list, str, pandas.Series or pandas.DataFrame, optional*) One or more templates or template titles to limit the search by.
- __title__: (*str, optional*) Record title to limit the search by.
- __keyword__: (*str or list, optional*) Keyword(s) to use for a string-based search of record content.  Only records containing all keywords will be returned. 
- __mongoquery__: (*str or dict, optional*) Mongodb find query to use in limiting searches by record element fields.  Note: only record parsing is supported, not field projection.
- __page__: (*int or None, optional*) If an int, then will return results only for that page of 10 records.  If None (default), then results for all pages will be compiled and returned.

Returns
- (*pandas.DataFrame*) All records matching the search request

Specify a template in the database to interact with

In [3]:
# Note: template should the name of a template in the database you are accessing!
template = 'FAQ'

Use query to fetch records

In [4]:
records = curator.query(template=template)
records

,id,template,workspace,user_id,title,xml_content,creation_date,last_modification_date,last_change_date,template_title
0,61e7d7734f87baec302de5c4,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,ref,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:58.407000+00:00,2021-08-26 13:15:58.407000+00:00,2021-08-26 13:15:58.715000+00:00,FAQ
1,61e7d77421ce6d3711a5aa7f,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,faq,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:57.438000+00:00,2021-08-26 13:15:57.438000+00:00,2021-08-26 13:15:57.750000+00:00,FAQ
2,61e7d7754f87baec302de5c8,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,formats,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:57.919000+00:00,2021-08-26 13:15:57.919000+00:00,2021-08-26 13:15:58.236000+00:00,FAQ
3,61e7d776023df5acc187b272,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,graphs,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:59.839000+00:00,2021-08-26 13:15:59.839000+00:00,2021-08-26 13:16:00.155000+00:00,FAQ
4,61e7d777e7c22f15811f3c51,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,lammps,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:59.365000+00:00,2021-08-26 13:15:59.365000+00:00,2021-08-26 13:15:59.674000+00:00,FAQ
5,61e7d778b7be9048c078d083,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,submit,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:56.939000+00:00,2021-08-26 13:15:56.939000+00:00,2021-08-26 13:15:57.264000+00:00,FAQ
6,61e7d7794f87baec302de5cc,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,manuscript,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:58.878000+00:00,2021-08-26 13:15:58.878000+00:00,2021-08-26 13:15:59.197000+00:00,FAQ


Pick the first record, and see its xml contents

In [5]:
record = records.iloc[0]
content = record.xml_content
print(content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I have downloaded an interatomic potential and wish to acknowledge the website in the references.  How do I do that?</question><answer>In addition to the citation information for the interatomic potential, please include "NIST Interatomic Potentials Repository: http://www.ctcms.nist.gov/potentials" in the references.</answer></faq>


## 3. Manage records

The methods associated with managing records requires having an account with the corresponding database and the correct privileges.

### 3.1. Upload a new record

New records can be uploaded to the database using the upload_record() method.

Parameters

- **template** (*str or pandas.Series*) The template or template title to associate with the record.
- **filename** (*str, optional*) Name of an XML file whose contents are to be uploaded.  Either filename or content required.
- **content** (*str or bytes, optional*) String content to upload. Either filename or content required.
- **title** (*str, optional*) Title to save the record as.  Optional if filename is given (title will be taken as filename without ext).
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being uploaded.
- **duplicatecheck** (*bool, optional*) If True (default), then a ValueError will be raised if a record already exists with the same template and title.  If False, no check is performed possibly allowing for multiple records with the same title to exist in the database.  Note: this check only searches the records that you have access to, so duplicates are possible from other users.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Use the content from the record queried above to upload a "test" record.

In [6]:
title = 'testrecord1'
curator.upload_record(template=template, title=title, content=content, verbose=True)

record testrecord1 (6307af837e0d3151f09d8bde) successfully uploaded.


Alternatively, the names of any local XML files can be specified and the method will automatically read the contents.  The file name without path and extension will be used for the title if no title is given.

Also, note that the workspace is being set to 'Global Public Workspace'.

In [7]:
# Save content to local file
filename = 'testrecord2.xml'
with open(filename, 'w') as f:
    f.write(content)
    
# Upload from file
curator.upload_record(template=template, filename=filename, verbose=True,
                      workspace='Global Public Workspace')

# Delete local file (keep working directory clean)
Path(filename).unlink()

record testrecord2 (6307af8787cb8a45bc5f7cf6) successfully uploaded.
record 6307af8787cb8a45bc5f7cf6 assigned to workspace 61e04376ea78175c20ac1554


### 3.2. Retrieve data records

The records that you own can be accessed using the get_records() and get_record() methods. get_records() will fetch all matching records, while get_record() will fetch a single record if exactly one match is found and will throw an error otherwise.

Parameters

- **template** (*str or pandas.Series, optional*) The template or template title to limit the search by.
- **title** (*str, optional*) The data record title to limit the search by.

In [8]:
curator.get_records(template=template)

,id,template,workspace,user_id,title,xml_content,creation_date,last_modification_date,last_change_date
0,6307af8787cb8a45bc5f7cf6,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,testrecord2,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2022-08-25 17:21:11.108000+00:00,2022-08-25 17:21:11.108000+00:00,2022-08-25 17:21:12.974000+00:00
1,6307af837e0d3151f09d8bde,61e7d772d2b0a949beaa4440,None,2,testrecord1,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2022-08-25 17:21:07.704000+00:00,2022-08-25 17:21:07.704000+00:00,2022-08-25 17:21:07.704000+00:00
2,61e7d776023df5acc187b272,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,graphs,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:59.839000+00:00,2021-08-26 13:15:59.839000+00:00,2021-08-26 13:16:00.155000+00:00
3,61e7d777e7c22f15811f3c51,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,lammps,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:59.365000+00:00,2021-08-26 13:15:59.365000+00:00,2021-08-26 13:15:59.674000+00:00
4,61e7d7794f87baec302de5cc,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,manuscript,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:58.878000+00:00,2021-08-26 13:15:58.878000+00:00,2021-08-26 13:15:59.197000+00:00
5,61e7d7734f87baec302de5c4,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,ref,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:58.407000+00:00,2021-08-26 13:15:58.407000+00:00,2021-08-26 13:15:58.715000+00:00
6,61e7d7754f87baec302de5c8,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,formats,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:57.919000+00:00,2021-08-26 13:15:57.919000+00:00,2021-08-26 13:15:58.236000+00:00
7,61e7d77421ce6d3711a5aa7f,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,faq,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:57.438000+00:00,2021-08-26 13:15:57.438000+00:00,2021-08-26 13:15:57.750000+00:00
8,61e7d778b7be9048c078d083,61e7d772d2b0a949beaa4440,61e04376ea78175c20ac1554,2,submit,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-08-26 13:15:56.939000+00:00,2021-08-26 13:15:56.939000+00:00,2021-08-26 13:15:57.264000+00:00


In [9]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                 6307af8787cb8a45bc5f7cf6
template                                           61e7d772d2b0a949beaa4440
workspace                                          61e04376ea78175c20ac1554
user_id                                                                   2
title                                                           testrecord2
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
creation_date                              2022-08-25 17:21:11.108000+00:00
last_modification_date                     2022-08-25 17:21:11.108000+00:00
last_change_date                           2022-08-25 17:21:12.974000+00:00
Name: 0, dtype: object


In [10]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I have downloaded an interatomic potential and wish to acknowledge the website in the references.  How do I do that?</question><answer>In addition to the citation information for the interatomic potential, please include "NIST Interatomic Potentials Repository: http://www.ctcms.nist.gov/potentials" in the references.</answer></faq>


### 3.3. Update an existing record

The content of a record in the database can be changed using the update_record() method.  Note that this only changes the record's content and all other metadata (user, workspace, database id,...) will remain unchanged.

- **record** (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the template and title parameters are ignored if given. Can contain the new conte
- **template** (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- **title** (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- **filename** (*str or Path, optional*) Path to file containing the new record content to upload. Either filename or content required.
- **content** (*str or bytes, optional*) New content to upload. Either filename or content required.
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being updated.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Change the content of testrecord2 by replacing "download" with "find"

In [11]:
new_content = record.xml_content.replace('download', 'find')
print(new_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I have finded an interatomic potential and wish to acknowledge the website in the references.  How do I do that?</question><answer>In addition to the citation information for the interatomic potential, please include "NIST Interatomic Potentials Repository: http://www.ctcms.nist.gov/potentials" in the references.</answer></faq>


In [12]:
curator.update_record(record=record, content=new_content, verbose=True)

record testrecord2 (6307af8787cb8a45bc5f7cf6) has been updated.


Retrieve the record again from the database showing nothing has changed except modification date and content

In [13]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                 6307af8787cb8a45bc5f7cf6
template                                           61e7d772d2b0a949beaa4440
workspace                                          61e04376ea78175c20ac1554
user_id                                                                   2
title                                                           testrecord2
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
creation_date                              2022-08-25 17:21:11.108000+00:00
last_modification_date                     2022-08-25 17:21:18.603000+00:00
last_change_date                           2022-08-25 17:21:18.608000+00:00
Name: 0, dtype: object


In [14]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I have finded an interatomic potential and wish to acknowledge the website in the references.  How do I do that?</question><answer>In addition to the citation information for the interatomic potential, please include "NIST Interatomic Potentials Repository: http://www.ctcms.nist.gov/potentials" in the references.</answer></faq>


### 3.4. Assign records to a workspace

The upload_record() and update_record() methods give the option to assign each record to a workspace if wanted.  After a record is added, the workspace can also be changed using the assign_records() method.

Parameters

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the records to.
- __records__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected records to assign to the workspace.  Cannot be given with ids, template, or title.
- __ids__ (*str or list, optional*) The ID(s) of the records to assign to the workspace.  Selecting records using ids has the least overhead. Cannot be given with records, template, or title.
- __template__ (*str or pandas.Series, optional*) The template or template title of records to assign to the workspace.  Cannot be given with records or ids.
- __title__ (*str, optional*) The title of a record to assign to the workspace. Cannot be given with records or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [15]:
curator.assign_records(workspace='Global Public Workspace', title='testrecord1', template=template, verbose=True)

record 6307af837e0d3151f09d8bde assigned to workspace 61e04376ea78175c20ac1554


### 3.5. Delete a record

A record can also be deleted by the assigned user using the delete_record() method.

Parameters

- __record__ (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the other parameters are ignored.
- __template__ (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- __title__ (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [16]:
# Delete by identifying with title + template
curator.delete_record(title='testrecord1', template=template, verbose=True)

record testrecord1 (6307af837e0d3151f09d8bde) has been deleted.


In [17]:
# Get a record first
record = curator.get_record(title='testrecord2', template=template)

# Delete by passing Series
curator.delete_record(record, verbose=True)

record testrecord2 (6307af8787cb8a45bc5f7cf6) has been deleted.


## 4. Manage blobs (raw files)

The database can also store any non-record files as blob files that can later be retrieved and downloaded all at once.

Create blob file for testing

In [18]:
filename = 'test_blob.txt'

with open(filename, 'w') as f:
    f.write('This is my blob for testing')

### 4.1. Upload blob

Blobs can be added to the database using the upload_blob() method.  When a record is uploaded, it will be associated with the given filename (minus the path).

Parameters

- __filename__ (*str or Path*) The path/name of the file to upload.
- __blobbytes__ (*bytesIO, optional*) Pre-loaded file contents.  Allows for the contents of open file-like objects to be passed in and associated with the filename.  If not given, then the file filename will be read instead.
- __workspace__ (*str or pandas.Series, optional*) If given, the blob will be assigned to this workspace after successfully being uploaded.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Returns

- __handle__ (*str*) The URL that can be used to retrieve the blob's contents.

In [19]:
handle = curator.upload_blob(filename=filename, verbose=True)
print(handle)

File "test_blob.txt" uploaded as blob "test_blob.txt" (6307af9c8a7c6e38a12361fb)
https://potentials.nist.gov/rest/blob/download/6307af9c8a7c6e38a12361fb/


### 4.2 Find blob metadata

The list of all blobs currently in the database, or all with an associated filename can be explored with the get_blobs() method.

Parameters

- __filename__ (*str, optional*) The name of the file to limit the search by.

A single blob can also be retrieved using the get_blob() method based on a filename, if it is unique in the database, or a database id.

Parameters

- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with filename
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with id.

__NOTE__: Exploring blob metadata appears to behave differently for some versions of CDCS, with the filename searches only showing blobs that the user owns, while id searches shows any blobs available to the user, either by being owned or in an available workspace.

- Identifying by filename only works for user blobs
- Identifying by id works for all blobs available to you (user + allowed workspaces)

In [20]:
curator.get_blobs()

,id,user_id,filename,handle,upload_date,pid
0,6307af9c8a7c6e38a12361fb,2,test_blob.txt,https://potentials.nist.gov/rest/blob/download...,2022-08-25 17:21:32+00:00,https://potentials.nist.gov/pid/rest/local/pot...


In [21]:
blobdata = curator.get_blob(filename=filename)
print(blobdata)

id                                      6307af9c8a7c6e38a12361fb
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download...
upload_date                            2022-08-25 17:21:32+00:00
pid            https://potentials.nist.gov/pid/rest/local/pot...
Name: 0, dtype: object


In [22]:
curator.get_blob(id=blobdata.id)

id                                      6307af9c8a7c6e38a12361fb
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download...
upload_date                            2022-08-25 17:21:32+00:00
pid            https://potentials.nist.gov/pid/rest/local/pot...
dtype: object

### 4.3 Retrieve blob contents

The contents of a blob can then be retrieved using the get_blob_contents() method.

Paramters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.

Returns

- __content__ (*bytes*) The blob's bytes contents.

In [23]:
print(curator.get_blob_contents(filename=filename))

b'This is my blob for testing'


Alternatively, the blob can be saved directly to a local file based on its associated filename with the download_blob() method.

Parameters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __savedir__ (*str or Path, optional*) The directory to save the file to.  Default value uses the current working directory.

In [24]:
curator.download_blob(filename=filename)

### 4.4. Assign blobs to a workspace

Just like records, blobs can also be assigned to workspaces.  This can be done during uploading, or it can be done after uploading using the assign_blobs() method.

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the blobs to.
- __blobs__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected blobs to assign to the workspace.  Cannot be given with ids or filename.
- __ids__ (*str or list, optional*) The ID(s) of the blobs to assign to the workspace.  Selecting blobs using ids has the least overhead. Cannot be given with blobs or filename.
- __filename__ (*str, optional*) The name of the blob file to assign to the workspace.  Cannot be given with blobs or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [25]:
curator.assign_blobs(workspace='Global Public Workspace', filename=filename, verbose=True)

blob 6307af9c8a7c6e38a12361fb assigned to workspace 61e04376ea78175c20ac1554


### 4.5. Delete blob

Finally, blobs can also be deleted using the delete_blob() method.

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [26]:
curator.delete_blob(filename=filename, verbose=True)

Successfully deleted blob "test_blob.txt" (6307af9c8a7c6e38a12361fb)


In [27]:
Path(filename).unlink()

## 5. Managing workspaces

Workspaces are used to manage access to content stored in a CDCS database. Records and blobs can be assigned to a workspace, then access control is set at the workspace level to all associated content.  The workspace assignment typically falls into three categories:

1. Content not assigned to a workspace will only be accessible to the user who created it.
2. Content assigned to the "Global Public Workspace" can be accessible to anyone who has read access to the database.
3. Alternate workspaces can be used to limit access to some content for only specific users.

### 5.1. Finding workspace metadata

Metadata for the available workspaces can be explored using the get_workspaces() and get_workspace() methods.

Parameters

title: (str, optional) The workspace title to limit the search by.

In [28]:
curator.get_workspaces()

,id,title,owner,is_public
0,61e04376ea78175c20ac1554,Global Public Workspace,None,True


In [29]:
curator.get_workspace(title='Global Public Workspace')

id           61e04376ea78175c20ac1554
title         Global Public Workspace
owner                            None
is_public                        True
Name: 0, dtype: object

The attribute global_workspace also retrieves the Global Public Workspace 

In [30]:
workspace = curator.global_workspace
print(workspace)

id           61e04376ea78175c20ac1554
title         Global Public Workspace
owner                            None
is_public                        True
Name: 0, dtype: object


## 6. Managing templates

Templates are used to categorize the types of records that a database stores and to validate uploaded record contents against a schema.  Templates are managed differently than other types of CDCS content in that they are version controlled; changes to template contents do not overwrite the current contents but rather get uploaded as a new template version.  Separate entries called template managers then specify which template versions exist, are active/disabled, and which version is the current version. 

### 6.1. Finding template metadata

A list of all available templates can be retrieved using the template_titles attribute.

In [31]:
curator.template_titles

['Potential',
 'relaxed_crystal',
 'FAQ',
 'Action',
 'PotentialProperties',
 'Request',
 'free_surface',
 'stacking_fault',
 'point_defect',
 'crystal_prototype',
 'potential_LAMMPS',
 'Citation',
 'calculation_diatom_scan',
 'calculation_isolated_atom',
 'potential_LAMMPS_KIM',
 'calculation_bond_angle_scan',
 'dislocation',
 'calculation_E_vs_r_scan',
 'calculation_relax_box',
 'calculation_relax_dynamic',
 'calculation_relax_static']

Metadata for the available templates can be explored using the get_templates() and get_template() methods.

Parameters

- __title__ (*str, optional*): The template title to limit the search by.
- __is_disabled__ (*bool, optional*): If True, then disabled templates will be returned.  If False (default), then active templates will be returned.
- __current__ (bool, optional*): If True (default), only current template versions will be returned.
- __useronly__ (*bool, optional*): If True, only a user's templates are returned. If False (default), then all global templates are returned.

In [32]:
curator.get_templates()

,id,user,filename,content,hash,dependencies,title
0,62dae7b074127dcccced663c,None,Potential.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",bfac108952e884965fe5e1517204b8f38d32b89f,[],Potential
1,62def2cf74127dcccced6958,None,relaxed_crystal.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",e3dd6b141935f0cfbe3bea611df3467d027658a7,[],relaxed_crystal
2,61e7d772d2b0a949beaa4440,None,record-interatomic-potential-faq.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",2dc0be555da041f782a7435c377ea7a20f28d3cd,[],FAQ
3,61e7d77ad2b0a949beaa4442,None,record-interatomic-potential-action.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",f1c3de8a64553d81bd10cfc0cb24dd3cc129c710,[],Action
4,61e7d8508e7c5e31a92bb296,None,record-per-potential-properties.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",ea7eeed57e0cea9b15dbda4b32c08266201a7e5a,[],PotentialProperties
5,61e7dc6f4f87baec302de99f,None,Request3,"<xsd:schema xmlns:xsd=""http://www.w3.org/2001/...",d108bacceb44a2789544ade71018b7f5ceacca1b,[],Request
6,62deddf80fe1c7fb407431c5,None,free_surface.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",3d898fb080654874a30166b22d7850ff4c5327af,[],free_surface
7,62dede62f63fed732aa874c0,None,stacking_fault.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",c60975939fff88aaea384494b901e11f366137c1,[],stacking_fault
8,62dede82567a57d4dfb2e1f5,None,point_defect.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",b077b95055d04a914e0d2f69f82fd47be562a5b5,[],point_defect
9,62d999e374127dcccced63ba,None,crystal_prototype.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",a711a977750c1ce42bb6dc35e3e71cfcb25d0304,[],crystal_prototype


In [33]:
curator.get_template('Citation')

id                                       61e7df98023df5acc187b36f
user                                                         None
filename                record-interatomic-potential-citation.xsd
content         <?xml version="1.0" encoding="UTF-8" standalon...
hash                     893099854c8ac392bad133ce71ac34f60407ab6f
dependencies                                                   []
title                                                    Citation
Name: 0, dtype: object

The get_template_managers() method fetches metadata associated with the various versions of templates in the database. 

Parameters

- __title__ (*str, optional*) The template title to limit the search by.
- __is_disabled__ (*bool, optional*) If True, then disabled templates will be returned.  If False (default), then active templates will bereturned.
- __useronly__ (*bool, optional*) If True, only a user's templates are returned. If False (default), then all global templates arereturned.

In [34]:
curator.get_template_managers('Citation')

,id,title,user,versions,current,is_disabled,disabled_versions,_cls
0,61e7df98023df5acc187b370,Citation,None,[61e7df98023df5acc187b36f],61e7df98023df5acc187b36f,False,[],VersionManager.TemplateVersionManager


### 6.2. Adding and modifying templates

__NOTE__: The upload and update operations for templates differ from other types of CDCS contents due to the template version control system.  Notably, the content associated with an uploaded template cannot be modified or replaced, simply new versions uploaded and managed.

New templates can be added using upload_template(). This will create a template manager for the template title as well as the first version of that template.

Parameters

- __filename__ (*str, optional*) Name of the XSD schema file to upload for the template.  Optional if title is given (filename will be taken as "title".xsd).
- __content__ (*str or bytes, optional*) String contents of an XSD schema file to upload for the template.  Optional if filename is given as a full path to the XSD file.
- __title__ (*str, optional*) Title to save the template as.  Optional if filename is given (title will be taken as filename without ext).
- __useronly__ (*bool, optional*) If True, the template will be associated only with the user. If False (default), it will be made a global template.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Calling update_template() will create a new version of a template and associate it with an existing template title.  Options allow for some automatic version management of the new/old templates.

Parameters
    
- __filename__ (*str, optional*) Name of the XSD schema file to upload for the template.  Optional if title is given (filename will be taken as "title".xsd).
- __content__ (*str or bytes, optional*) String contents of an XSD schema file to upload for the template.  Optional if filename is given as a full path to the XSD file.
- __title__ (*str, optional*) Title to save the template as.  Optional if filename is given (title will be taken as filename without ext).
- __template_manager__ (*pandas.Series, optional*) Can be given instead of title if the template_manager info has already been retrieved from the database.
- __set_current__ (*bool, optional*) If True (default), will set the uploaded version of the template to be the current active version.
- __disable_old__ (*bool, optional*) If True, any active versions of the template besides the newly created and the current version (if different) will be disabled.  Default value is False.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

### 6.3. Template version management operations

As templates cannot be modified or deleted, there are a number of operations that allow for template versions and managers to be made less accessible.

Individual template versions can be set as disabled or returned to active status using disable_template() and restore_template(), respectively.  Disabling a template version prevents users from adding records to that template version.  For each template title, a single template version is considered the "current" version. The set_current_template() method changes which template version is the current one.  All three of these methods rely on parameters that identify a specific version of a template.

Parameters
    
- __title__ (*str, optional*) The template title.
- __version__ (*int, optional*) The version of the template to make current.  Required unless template_id is given.  Note that version numbers start at 1.
- __template_manager__ (*pandas.Series, optional*) Template manager information for the template.  This can be given instead of title to avoid querying for the template manager.
- __template_id__ (*str, optional*) The database id for the template to set as current.  If given, then no other parameters are allowed (or needed).
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Additionally, all versions of a given template can be disabled or restored using disable_template_manager() and restore_template_manager(), respectively.  The parameters for these methods are used to identify the template family to disable/restore. 

Parameters
    
- __title__ (*str, optional*) The template title.
- __template_manager__ (*pandas.Series, optional*) Template manager information for the template.  This can be given instead of title to avoid querying for the template manager.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

### 6.4. Template migration and validation

Each record is associated with a specific template. Migration allows for records to be associated with a different template instead, as long as the record contents can be validated against the new template's schema.  This type of operation is typically done when a new version of a template is added that extends the current template.  Validation performs the schema validation checks for the records against the new template without migrating the records to the new template.

These operations Will hopefully be added soon as the proper use of the REST calls are being worked out. 

## 7. Managing XSLTs 

XSLT files allow for the XML content of records to be transformed. This is used by CDCS for a variety of purposes
- Transformations to HTML can be used to render the contents of record entries into user-friendly web pages.
- Transformations can also be specified to convert the record contents into other text-based formats that users may wish to download.
- A transformation can also be used for data migration when a new template version is not backwards compatible with the current template version.  Here, the transformation would be XML to XML so that the content is valid with the new template schema. 

### 7.1. Exploring existing XSLTs

Similar to other content types, get_xslts() and get_xslt() fetch all or one matching XSLT files from the database.  Each XSLT can be identified by its name or filename.

Parameters
 
- __name__ (*str, optional*) The xslt name to limit the search by.
- __filename__ (*str, optional*) The xslt filename to limit the search by.

### 7.2. Uploading a new XSLT

New XSLTs can be uploaded using upload_xslt().

Parameters
    
- __name__ (*str, optional*) The name to associate with the XSLT file.  Optional if filename is given as name will be taken as filename without its extension.
- __filename__ (*str, optional*) The filename to associate with the XSLT file.  Optional if name and content are given.  If not given, filename will be set to name + '.xsl'. Will read the file contents if the file exists and content is not given.
- __content__ (*str or bytes, optional*) XSLT file contents.  Optional if filename is given and points to a file that exists.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

### 7.3. Modifying an existing XSLT

Any uploaded XSLT entries can be modified and updated using update_xslt(). Note that there are additional parameters compared to other update operations as the name and filename fields can be updated in addition to the content.

Parameters
    
- __name__ (*str, optional*) An xslt name. Will be used to identify the existing xslt entry to update if neither xslt nor xslt_id parameters are given. If either xslt or xslt_id are given, this can be used to assign a new name to the entry.
- __filename__ (*str, optional*) An xslt filename. Will be used to identify the existing xslt entry to update if neither xslt nor xslt_id parameters are given. If either xslt or xslt_id are given, this can be used to assign a new filename to the entry.
- __content__ (*str, optional*) New xsl content to assign to the entry.
- __newname__ (*str, optional*) New name to assign to the entry.
- __newfilename__ (*str, optional*) New filename to assign to the entry.
- __xslt__ (*pd.Series, optional*) The xslt entry information for the entry that is being updated.  
- __xslt_id__ (*str, optional*) The database id that uniquely identifies the xslt entry. 
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

### 7.4. Deleting an existing XSLT

Any XSLT files can also be removed from the database using delete_xslt().

Parameters
    
- __name__ (*str, optional*) Values for name and filename can be specified to try to uniquely identify the XSLT entry to delete.  Cannot be combined with xslt or xslt_id as they uniquely identify the XSLT on their own.
- __filename__ (*str, optional*) Values for name and filename can be specified to try to uniquely identify the XSLT entry to delete.  Cannot be combined with xslt or xslt_id as they uniquely identify the XSLT on their own.
- __xslt__ (*pd.Series, optional*) The xslt entry information for the entry that is to be deleted.  Cannot be combined with name, filename or xslt_id.
- __xslt_id__ (* str, optional*) The database id that uniquely identifies the xslt entry to delete.  Cannot be combined with name, filename or xslt.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

## 8. Managing PID XPATHs

To help make data in a CDCS more compliant with FAIR practices, a PID (persistent identifier) can be assigned to each record entry and blob.  Without a PID, the URL for accessing a record/blob contains the database ID that was assigned to that content. The database ID is not persistent as deleting and reuploading the content will result in it being assigned a different database ID. This is especially relevant when restoring a database from a backup as the database IDs are not retained.

The PID XPATH operations listed here make it easy for PIDs to be automatically assigned to records of a given schema.  The steps involved are

1. The CDCS database needs to be set up to allow for automatically assigning PIDs, and one or more domains needs to be defined.  Contact whoever is in charge of hosting your CDCS instance as these options either require high administration privileges or are not accessible with REST commands or from the web interface.
2. In order for a PID to be assigned to a record, the PID must be included in the record's XML content as a full URL.  The database automatically finds the PID URL field from the XML content based on an XPATH that is set on a per-template basis.  The methods described in this section allow for these template-specific PID XPATHs to be managed.
3. Each PID URL will be of the form {host}/pid/rest/{provider}/{domain}/{record}.  Host is the host URL for the database, provider is typically "local", domain is one of the domains defined in step #1, and record then is some record-specific ID that makes the full URL unique.  As long as this PID URL is in the appropriate XML field specified by the PID XPATH for the template, is consistent with the form described, and is unique, then the record can be accessed from the PID URL.

### 8.1. Exploring existing PID XPATHs


The currently set PID XPATHs can be retrieved and explored with get_pid_xpaths() and get_pid_xpath().

Parameters

- __template__ (*str or pandas.Series, optional*) The template or template title to limit the search by.

### 8.2. Uploading and updating PID XPATHS

PID XPATHs can be set and changed with upload_pid_xpath() and update_pid_xpath(), respectively.
    
Parameters
    
- __template__ (*str or pandas.Series*) The template or template title.
- __xpath__ (*str*) The xpath to use for the pid field for the template.

### 8.3. Deleting PID XPATHS

A set PID XPATH can be deleted with delete_pid_xpath().
    
Parameters
    
- __template__ (*str or pandas.Series*) The template or template title.